In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

在Vertex AI模型注册表上部署BiqQuery ML模型并进行预测

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_registry/bqml_vertexai_model_registry.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_registry/bqml_vertexai_model_registry.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_registry/bqml_vertexai_model_registry.ipynb" target="_blank">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

## 概述

本教程演示了如何使用BigQuery ML训练模型并上传到Vertex AI模型注册表，然后进行批量预测。

了解更多关于[Vertex AI模型注册表](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)和了解更多关于[BigQuery ML](https://cloud.google.com/vertex-ai/docs/beginner/bqml)。

### 目标

在本教程中，您将学习如何使用`Vertex AI Model Registry`与`BigQuery ML`，并进行批量预测。

本教程使用以下Google Cloud ML服务和资源：

- `Vertex AI Model Registry`
- `Vertex AI Endpoint`资源
- `Vertex Explainable AI`
- `BigQuery ML`


执行的步骤包括：

- 使用`BigQuery ML`训练模型
- 将模型上传到`Vertex AI Model Registry`
- 创建一个`Vertex AI Endpoint`资源
- 部署`Model`资源到`Endpoint`资源
- 向模型端点发出`prediction`请求
- 在`Model`资源上运行`batch prediction`任务

数据集

本教程使用的数据集是来自<a href="https://cloud.google.com/bigquery/public-data" target="_blank">BigQuery公共数据集</a>的企鹅数据集。该数据集版本用于根据可用特征如钩鼻长度、鳍深度等预测企鹅的物种。

### 成本

本教程使用谷歌云的可计费组件:

* Vertex AI
* BigQuery ML

了解<a href="https://cloud.google.com/vertex-ai/pricing" target="_blank">Vertex AI 价格</a>和<a href="https://cloud.google.com/bigquery/pricing" target="_blank">BigQuery 价格</a>，并使用<a href="https://cloud.google.com/products/calculator/" target="_blank">价格计算器</a>
根据您的预期使用量生成成本估算。

### 安装额外的包

安装以下要求执行此笔记本所需的包。

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform \
                         google-cloud-bigquery \
                         pyarrow --quiet

只有Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"
DATA_REGION = "US"

将您的Google Cloud账户进行身份验证

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

**1. Vertex AI Workbench**
*不用做任何操作，因为您已经通过身份验证。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务帐户或其他
* 请查看如何授予您的服务帐户云存储权限, 可在以下链接查看 https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### 导入库

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery

### 初始化用于Python的Vertex AI和BigQuery SDK

为您的项目和对应的存储桶初始化Python的Vertex AI和BigQuery SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bqclient = bigquery.Client(project=PROJECT_ID)

BigQuery ML介绍

BigQuery ML（BQML）提供了使用SQL语法在BigQuery中训练ML表格模型（如分类和回归）的能力。

了解更多关于<a href="https://cloud.google.com/bigquery-ml/docs" target="_blank">BigQuery ML文档</a>。

用于训练的 BigQuery 表

In [ ]:
# Define BigQuery table to be used for training

BQ_TABLE = "bigquery-public-data.ml_datasets.penguins"

### 创建 BigQuery 数据集资源
首先，在您的项目中创建一个空的数据集资源。

In [ ]:
BQ_DATASET_NAME = "penguins"

# if dataset exists, delete it
! bq rm -r -f $PROJECT_ID:$BQ_DATASET_NAME

DATASET_QUERY = f"""CREATE SCHEMA {BQ_DATASET_NAME}"""
job = bqclient.query(DATASET_QUERY)
job.result()
print(job.state)

## 训练 BigQuery ML 模型并上传到 Vertex AI 模型注册表
接下来，您将从公共数据集 penguins 创建并训练一个 `BigQuery ML` 表格回归模型，并使用 `CREATE MODEL` 语句将模型存储在您的项目 `Vertex AI 模型注册表` 中。模型配置在 `OPTIONS` 语句中指定如下：

- `model_type`：要训练的表格模型的类型和架构，例如 LOGISTIC_REG。

- `labels`：作为标签的列。

- `model_registry`：要将 BigQuery ML 模型注册到 Vertex AI 模型注册表，您必须使用 `model_registry="vertex_ai"`。

在此阶段，如果您正在使用受支持的模型类型，BQML 会将与模型相关的元数据导出到 Vertex，以便运行 Vertex Explainable AI。了解更多关于<a href="https://cloud.google.com/bigquery-ml/docs/vertex-xai" target="_blank">支持的 BQML 模型类型</a>。

了解更多有关<a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create" target="_blank">CREATE MODEL 语句</a>。

了解更多有关<a href="https://cloud.google.com/bigquery-ml/docs/managing-models-vertex" target="_blank">在 Vertex AI 模型注册表中管理 BigQuery ML 模型</a>。

In [ ]:
# Write the query to create Big Query ML model

MODEL_NAME = "penguins-lr"
MODEL_QUERY = f"""
CREATE OR REPLACE MODEL `{BQ_DATASET_NAME}.{MODEL_NAME}`
OPTIONS(
    model_type='LOGISTIC_REG',
    labels = ['species'],
    model_registry='vertex_ai'
    )
AS
SELECT *
FROM `{BQ_TABLE}`
"""

### 创建 BigQuery ML 模型
使用上面的查询和之前创建的 BigQuery 客户端来创建 BigQuery ML 模型。

In [ ]:
# Run the model creation query using BigQuery client

job = bqclient.query(MODEL_QUERY)
print(f"Job state: {job.state}\nJob Error:{job.errors}")

检查工作状态：

In [ ]:
job.result()
print(job.state)

### 在 Vertex Model Registry 中找到模型

您可以使用`Vertex AI Model()`方法，通过`model_name`参数找到自动注册的模型。

In [ ]:
model = aiplatform.Model(model_name=MODEL_NAME)

print(model.gca_resource)

将Vertex AI模型资源部署到Vertex AI终端资源
在模型可用于提供在线预测之前，您必须将模型部署到一个`终端`；部署模型会将物理资源与模型关联起来，以便以低延迟提供在线预测。

了解更多关于<a href="https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api#aiplatform_deploy_model_custom_trained_model_sample-python" target="_blank">使用Vertex AI API部署模型</a>

### 创建一个Vertex AI端点资源

如果您要部署模型到现有端点，可以跳过这个部分。

- `display_name`: 端点的显示名称。
- `project`: 您正在创建端点的项目ID。
- `location`: 您正在使用Vertex AI的区域。

In [ ]:
ENDPOINT_DISPLAY_NAME = "bqml-lr-model-endpoint"

endpoint = aiplatform.Endpoint.create(
    display_name=ENDPOINT_DISPLAY_NAME,
    project=PROJECT_ID,
    location=REGION,
)

print(endpoint.display_name)
print(endpoint.resource_name)

### 将Vertex AI模型资源部署到Vertex AI端点资源

In [ ]:
DEPLOYED_NAME = "bqml-lr-penguins"

model.deploy(endpoint=endpoint, deployed_model_display_name=DEPLOYED_NAME)

print(model.display_name)
print(model.resource_name)

发送预测请求到Vertex AI端点资源

现在您的Vertex AI模型资源已部署到Vertex AI `端点`资源，您可以通过向`端点`资源发送预测请求来进行在线预测。

了解更多关于<a href="https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models" target="_blank">从自定义训练模型获取在线预测</a>

In [ ]:
instance = {
    "island": "Dream",
    "culmen_length_mm": 36.6,
    "culmen_depth_mm": 18.4,
    "flipper_length_mm": 184.0,
    "body_mass_g": 3475.0,
    "sex": "FEMALE",
}

prediction = endpoint.predict([instance])
print(prediction)

向Vertex AI终端点资源发送可解释的AI请求

如果这是支持的BQML模型类型，您可以为您的资源获取解释。要了解有关在BQML模型上使用Vertex AI可解释性的更多信息，请参阅[BigQuery ML模型的可解释AI](https://cloud.google.com/bigquery-ml/docs/vertex-xai)。

In [ ]:
instance = {
    "island": "Dream",
    "culmen_length_mm": 36.6,
    "culmen_depth_mm": 18.4,
    "flipper_length_mm": 184.0,
    "body_mass_g": 3475.0,
    "sex": "FEMALE",
}

explanation = endpoint.explain([instance])
print(explanation)

## 在 BigQuery ML 模型上进行批量预测
在这里，您可以直接从 BigQuery ML 模型请求批量预测；您无需部署模型到端点。对于支持批量和在线预测的数据类型，在您不需要立即响应并且希望通过单个请求处理累积数据时，请使用批量预测。

了解更多关于 <a href="https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-predict" target="_blank">ML.PREDICT 函数</a>

In [ ]:
sql_ml_predict = f"""SELECT * FROM ML.PREDICT(MODEL `{PROJECT_ID}.{BQ_DATASET_NAME}.{MODEL_NAME}`, 
(SELECT
      *
    FROM
      `{BQ_TABLE}` LIMIT 10))"""

job = bqclient.query(sql_ml_predict)
prediction_result = job.result().to_arrow().to_pandas()

In [ ]:
display(prediction_result.head())

清理

要清理此项目中使用的所有Google Cloud资源，您可以<a href="https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects" target="_blank">删除用于本教程的Google Cloud项目</a>。

了解有关<a href="https://cloud.google.com/bigquery-ml/docs/managing-models-vertex" target="_blank">从Vertex AI Model Registry中删除BigQuery ML模型</a>的更多信息

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Delete the endpoint using the Vertex endpoint object
endpoint.undeploy_all()
endpoint.delete()

In [ ]:
# Delete BigQuery ML model

delete_query = f"""DROP MODEL `{PROJECT_ID}.{BQ_DATASET_NAME}.{MODEL_NAME}`"""
job = bqclient.query(delete_query)
job.result()

In [ ]:
# Delete the created BigQuery dataset
! bq rm -r -f $PROJECT_ID:$BQ_DATASET_NAME